In [ ]:
'''
Project Euler Problem 083: Path Sum: Four Ways
'''

In [ ]:
import csv
from random import sample

with open('euler_ressources/0083_matrix.txt') as csvfile:
    matrix = [list(map(int, row)) for row in csv.reader(csvfile, delimiter=',')]

matrix_small = [[131, 673, 234, 103, 18], 
    [201, 96, 342, 965, 150],
    [630, 803, 746, 422, 111],
    [537, 699, 497, 121, 956],
    [805, 732, 524, 37, 331]]

In [ ]:
# This function DOES NOT WORK!!!

def four_ways(matrix):
    N = len(matrix)

    sol = [[0] * N for _ in range(N)]
    sol[0][0] = matrix[0][0]
    
    for s in range(1, N):
        ls = 2*s
        vec_mat = [matrix_small[min(k, s)][min(ls-k, s)] for k in range(0, ls+1)]
        vec_sol = [sol[min(k, s-1)][min(ls-k,s-1)] for k in range(0, ls+1)]
    
        for t in range(ls+1):
            if t == 0:
                sol[0][s] = min([sum([vec_mat[l] for l in range(k+1)])+vec_sol[k] for k in range(ls+1)])
            elif t < s:
                i, j = t, s
                sol[i][j] = min(min([sum([vec_mat[l] for l in range(t, k+1)])+vec_sol[k] for k in range(t, ls+1)]), vec_mat[t]+sol[i-1][j])
            elif t == s:
                sol[s][s] = min(min([sum([vec_mat[l] for l in range(t, k+1)])+vec_sol[k] for k in range(t+1, ls+1)]), vec_mat[t]+sol[s-1][s])
            elif t > s:
                i, j = s, ls - t
                sol[i][j] = min(min([sum([vec_mat[l] for l in range(t, k+1)])+vec_sol[k] for k in range(t, ls+1)]), vec_mat[t]+sol[i][j+1])

    print(sol)
    


In [ ]:
def four_ways_dijkstra(matrix):
    N = len(matrix)
    maxentry = max(max(matrix, key=lambda x: max(x)))
    INF = N*N*maxentry
    
    indices = list(range(N))

    nodes = [(i, j) for i in range(N) for j in range(N)]

    dy = [1, -1, 0, 0]
    dx = [0, 0, -1, 1]

    weights = {(i, j): {(i+di, j+dj): matrix[i+di if i+di in indices else 0][j+dj if j+dj in indices else 0] 
        for di, dj in zip(dx, dy)} for i in range(N) for j in range(N)}
    
    weights = {k: {l: w for l, w in v.items() if l[0] in indices and l[1] in indices} for k, v in weights.items()}

    unvisited = {node: None for node in nodes}
    visited = {}
    previous = {}

    current = (0, 0)
    currentDistance = matrix[0][0]
    unvisited[current] = currentDistance

    while True:
        for neighbour, weight in weights[current].items():
            if neighbour not in unvisited: 
                continue
            newDistance = currentDistance + weight
            if unvisited[neighbour] is None or unvisited[neighbour] > newDistance:
                unvisited[neighbour] = newDistance
                previous[neighbour] = current
        visited[current] = currentDistance
        del unvisited[current]
        if not unvisited:
            break
        candidates = [node for node in unvisited.items() if node[1]]
        current, currentDistance = sorted(candidates, key = lambda x: x[1])[0]

    current = end = (N-1, N-1)
    path = [end]

    while True:
        current = previous[current]
        path += [current]
        if current == (0, 0):
            break

    return visited[end], path[::-1]


In [ ]:
N = 10
matrix_large = [sample(range(N**2), N) for _ in range(N)]

sol, path = four_ways_dijkstra(matrix_large)
print(f'The minimal path sum is {sol}.')
for pos in path[:-1]:
    print(pos, end=' -> ')
print(path[-1])
print(len(path))